# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1647039937771_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
# export
from pyspark.sql.functions import col
from pyspark.sql import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'ntalukder6'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    return trips.filter(trips.trip_distance >= 2.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
#export
def manhattan_trips(trips, lookup):
    
    trips = trips.withColumn("passenger_count", trips["passenger_count"].cast('int'))
    trips = trips.withColumn("DOLocationID", trips["DOLocationID"].cast('int'))
    trips = trips.join(lookup, trips.DOLocationID == lookup.LocationID).where(lookup.Borough == "Manhattan")
    
    return trips.select(['passenger_count','DOLocationID']).groupBy('DOLocationID').sum('passenger_count')\
            .sort(col('sum(passenger_count)').desc()).limit(20)\
            .withColumnRenamed("sum(passenger_count)", "pcount")
    
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
#     pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [7]:
#export
def weighted_profit(trips, mtrips): 
    
    
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
#     pass

#     trips_data_process = trips.select(['PULocationID', 'DOLocationID', 'total_amount'])
    
    

    #avg amount
    trips = trips.withColumn("total_amount", trips["total_amount"].cast('double'))
    trips = trips.withColumn("PULocationID", trips["PULocationID"].cast('int'))
    
    
#     average_amount = trips_data_process.groupBy('PULocationID').avg('total_amount')
    average_amount = trips.groupBy('PULocationID').avg('total_amount')

    #trips grouped by pick up location
    total_count_each_location = trips.groupBy('PULocationID').count().withColumnRenamed("count", "total_count")

#     total_count_each_location = trips_data_process.groupBy('PULocationID').count()


    top_values = [row[0] for row in mtrips.select('DOLocationID').collect()]
    
    masked_value = trips.DOLocationID.isin(top_values)
#     masked_value = trips_data_process.DOLocationID.isin(top_values)

#     trips_data_masked = trips_data_process[masked_value] #filter the whole data by top drop off locations
    trips_data_masked = trips[masked_value]
    
    
#     trips_data_masked = trips_data_masked.withColumn("PULocationID", trips_data_masked["PULocationID"].cast('int'))
    
    
    trips_ended_at_popular_location = trips_data_masked.groupBy('PULocationID').count().withColumnRenamed("count", "popular_count")
#     trips_ended_at_popular_location = trips_ended_at_popular_location.withColumnRenamed("count", "popular_count")

#     trips_ended_at_popular_location.show()
    proportion_dataframe = trips_ended_at_popular_location.join(total_count_each_location, "PULocationID", 'inner')
        
    proportion_dataframe = proportion_dataframe.withColumn("proportion", (col("popular_count") / col("total_count")))
#         .withColumn("proportion", (col("popular_count") / col("count")).cast('decimal(30,8)'))
        
        

    profit_dataframe = proportion_dataframe\
        .join(average_amount, "PULocationID", 'inner')
    
    profit_dataframe = profit_dataframe.withColumn("weighted_profit", (col("proportion")*col("avg(total_amount)")).cast('decimal(30,8)'))
#             .withColumn("weighted_profit", (col("proportion")*col("avg(total_amount)")))

    

    return profit_dataframe.select('PULocationID', 'weighted_profit')




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [8]:
#export
def final_output(wp, lookup): 
    
    
    
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    joined_output_df = wp.join(lookup, wp.PULocationID == lookup.LocationID, 'outer')
    return joined_output_df.select('Zone', 'Borough', 'weighted_profit').sort(col('weighted_profit').desc()).limit(20)

#     pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [9]:
def test_weighted(size):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
#     final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    wp.show()
    return wp
    
    # Writes out as a CSV to your bucket.
#     final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
weighted_df = test_weighted("small")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ntalukder6
Trip Count:  7667792
+------------+---------------+
|PULocationID|weighted_profit|
+------------+---------------+
|         148|     7.24102559|
|         243|     8.07104183|
|          65|     6.69655349|
|         133|     3.04276048|
|         126|     5.75015095|
|          81|     4.60678585|
|          76|     4.05649377|
|         192|     4.56424700|
|          12|    10.03800941|
|         209|     9.49565766|
|         122|     7.26495868|
|         140|     8.27149623|
|         177|     2.55338972|
|           1|     6.32938776|
|         206|    21.31508876|
|           6|    31.55555556|
|         205|     7.74124926|
|         142|     8.06871880|
|           3|     5.66724537|
|         178|     4.38795552|
+------------+---------------+
only showing top 20 rows

In [11]:
weighted_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PULocationID: integer (nullable = true)
 |-- weighted_profit: decimal(30,8) (nullable = true)

In [12]:
output_bucket = "s3://cse6242-ntalukder6/decimal-final"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
main("large", output_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ntalukder6
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|Flushing Meadows-...|       Queens|    30.34197174|
|        Baisley Park|       Queens|    28.91476313|
|     Randalls Island|    Manhattan|    26.10554401|
|       South Jamaica|       Queens|    25.61000645|
|        Astoria Park|       Queens|    21.41180544|
|             Maspeth|       Queens|    21.35670279|
|Briarwood/Jamaica...|       Queens|    20.55267043|
|Springfield Garde...|       Queens|    19.90214863|
|             Jamaica|       Queens|    19.54251898|
|         Kew Gardens|       Queens|    19.29679947|
|         Jamaica Bay|       Queens|    19.11349065|
|              Corona|       Queens|    17.83623531|
|   LaGuardia Airport|       Queens|    17.81139765|
|    South Ozone Park|       Queens|    17.48742072|
|         JFK Airport|       Queens|    17.31870744|
|Eltingville